# Function Handler

### 1.1 Introduction

LionAGI is an intelligent agent framework, it integrate big data with advanced Machine Learning models such as LLM. 

In this tutorial, we will go through **special function call handlers**

When handling big dataset, it is very common to use loops, but they can get complex quite quickly, and prone to errors. 
LionAGI provides a set of tools to handle various loops and other problems in function calling

Synchonous function call handlers:

- `to_list`  : Convert given object to list, additionally can flatten to 1-D and dropna
- `lcall`   : takes a list of inputs and apply a given function on each element 

Asynchonous function call handlers:

- `alcall` : Async List Call, similar to `lcall` but suitable for async
- `mcall`   : Mapped Call, map elements to functions, apply each function to corresponding element, or each element 
- `bcall`   : Batch Call, group elements into batches, apply function to each batch aysnchronously, and wait one batch to finish before next
- `tcall`   : Timed Call, delay, timeout, exception handling and calculates runtime
- `rcall`   : Retry Call, retry with backoff and default value

In [16]:
import lionagi.libs.ln_func_call as func_call

##### 1.1 `lcall` - list call

the first special function calling method is called l_call (list call), 

you can operate a single function on the whole set of input list to begin with, we will create a test input list and a test function

In [17]:
a = range(1,6)
f1 = lambda x: x**2

func_call.lcall(a, f1)

[1, 4, 9, 16, 25]

you can also pass \**kwargs of the functions

the input for lcall should be a single list, (you can bundle various inputs in objects)

In [18]:
def multiply_and_increment(x, factor=1, increment=0):
    print(x * factor + increment)

func_call.lcall(a, multiply_and_increment, factor=2, increment=3);

5
7
9
11
13


### 1.2 Async call handlers for Multi-Elements

LionAGI's core is designed to be async only, these handlers can help process large amounts of data concurrently, under control and rules. 

(sync functions also works)

there are many handlers for async operations

In [19]:
import asyncio

define an async function, and try to run it as usual 

##### 1.2.1 Syntax of Asynchronous Functions

In [20]:
async def add_1(x):
    return x+1

add_1(2)

<coroutine object add_1 at 0x118602b00>

This is the syntax of asynchronous functions, 

- if it is called without `await` keyword, it returns a coroutine object, meaning it is not executed rather packaged for future execution.
- if it is called with `await` keyword, it will wait for the function to finish executing and return the output. 

we need to add await in front of an async function to recieve the output, 
rather than a coroutine object

In [21]:
async def async_inverse(x):
    return 1 / x if x != 0 else None

await async_inverse(2)

0.5

##### 1.2.2 `alcall` - async list call

we can use alcall (async list call) to run an async function on a list

In [22]:
async def async_increment(x, inverse=False):
    if inverse:
        return await async_inverse(x+1)
    return x + 1

In [23]:
await func_call.alcall([[1, 3], [7, 9]], async_increment, inverse=True)

[0.5, 0.25, 0.125, 0.1]

##### 1.2.3 `mcall` - mapped call

allows you to map functions and arguments to a collection of objects, and call them all in parallel. 
- and apply each function to its corresponding input
- or apply each function to every input :  'explode'

In [24]:
f0 = lambda x: x+2
f1 = lambda x: x*2
f2 = lambda x: x**2

await func_call.mcall([3,4,5], [f0, f1, f2])

[5, 8, 25]

In [25]:
await func_call.mcall([3,4,5,6,7], [f0, f1, f2], explode=True)

[[5, 6, 7, 8, 9], [6, 8, 10, 12, 14], [9, 16, 25, 36, 49]]

##### 1.2.4 `bcall` - batch call

In [26]:
async def process_item(item):
    await asyncio.sleep(0.5)
    print(item*2)

inputs = [1, 2, 3, 4, 5]
batch_size = 2

# you should see the results get printed 2 items at a time, with a 0.5 second delay between each batch, 
# last batch should only have 1 element
await func_call.bcall(inputs, process_item, batch_size=2);

2
4
6
8
10


In [27]:
async def process_item_with_exception(item):
    if item == 3:
        raise ValueError("Error processing item")
    await asyncio.sleep(0.5)
    print(item*2)
    return item * 2

# 5 and 6 didn't get processed because the function stopped after the second batch due to the exception
inputs = [1, 2, 3, 4, 5, 6]
batch_size = 2
try:
    results = await func_call.bcall(inputs, process_item_with_exception, batch_size=2)
    print(results)
except ValueError as e:
    print(f"Caught an exception: {e}")

ERROR:root:Error in call_handler: Error processing item


2
4
Caught an exception: Error processing item


### 1.3 Async Call Handlers for single element

##### 1.3.1 `tcall` timed call

Handles both synchronous and asynchronous calls with optional delay, error handling, and execution timing.

In [28]:
async def async_function(input_):
    await asyncio.sleep(0.5)  # Simulate a delay
    if input_ == 'error':
        raise ValueError("Error triggered")
    return input_.upper()

# you can add a delay at the tcall
await func_call.tcall(
    func=async_function, input_='hello',
    delay=0.1, err_msg = 'Somehow Failed', 
    ignore_err=False, timing=True, timeout=None
)

8


('HELLO', 0.6023170948028564)

In [29]:
await func_call.tcall(
    func=async_function, input_='error',
    err_msg = 'Somehow Failed', ignore_err=True, 
)

Somehow Failed Error: Error triggered


##### 1.3.2 `rcall` - retry call

retry call is another function for exception handling during runtime 
- retries
- delay
- backoff_factor
- default
- timeout

In [30]:
try:
    await func_call.rcall(
        func=async_function, 
        input_='error',
        retries=3,
        backoff_factor=1,       # backoff factor 1, means we do not increase the wait time between retries
        delay=0.5               # the delay in rcall represents the time to wait after a failed attempt
    )
except Exception as e:
    print(f"Caught an exception: {e}")

An error occurred: Error triggered
An error occurred: Error triggered
An error occurred: Error triggered
Caught an exception: Error triggered
